### Setup env
This is notebook specific setup, my module path is different on my vm so this is a workaround

In [1]:
import sys
import os


path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)

### Load Dataset
You can change the type of motion by changing the filepath
Dataset right now just repeats the same motion 100 times

In [2]:
from diffusion.data_loaders.motion_dataset_v2 import MotionDataset
dataset = MotionDataset("data/motions/humanoid3d_walk.txt", shuffle=True)
len(dataset), dataset[0], dataset[0].trajectories.shape

Tmp angle [0.0, 0.0, 0.847532, 0.998678, 0.01410399999999995, 0.049422999999999995, -0.0006980000000000042, 0.019374995056800275, 0.008037254877450587, -0.09523902811084285, -0.0, 0.0, -0.0, -0.15553532463598624, 0.23919429256424163, 0.20739656997070288, 0.170571, 0.3529631848273465, -0.2610682953696931, -0.24560532144975333, 0.581348, 0.02035205257945668, -0.5175742452141794, -0.11376339039728192, -0.249116, 0.020556236630260034, -0.019534498786735962, 0.0655269812790598, -0.05606350142619236, 0.15209578259548684, 0.1827420948157945, -0.391532, 0.1931167851688944, -0.2978918547932108, -0.08305715225197069] 35
[1.007952 0.013846 0.024595]
[1.007952 0.013846]


(32,
 Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8475,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0415, -0.0047,  0.8466,  ...,  0.4419, -0.9102,  1.1832],
         [ 0.0817, -0.0105,  0.8483,  ...,  1.9768, -2.7716,  1.5138],
         ...,
         [ 0.9535,  0.0164,  0.8741,  ..., -0.0242,  0.5358,  0.5449],
         [ 0.9810,  0.0152,  0.8736,  ...,  0.0819,  0.5424,  0.2741],
         [ 1.0080,  0.0138,  0.8721,  ..., -0.0949,  0.5196,  0.1573]]), conditions={0: tensor([ 0.0000e+00,  0.0000e+00,  8.4753e-01,  9.9868e-01,  1.4104e-02,
          4.9423e-02, -6.9800e-04,  1.9375e-02,  8.0373e-03, -9.5239e-02,
         -0.0000e+00,  0.0000e+00, -0.0000e+00, -1.5554e-01,  2.3919e-01,
          2.0740e-01,  1.7057e-01,  3.5296e-01, -2.6107e-01, -2.4561e-01,
          5.8135e-01,  2.0352e-02, -5.1757e-01, -1.1376e-01, -2.4912e-01,
          2.0556e-02, -1.9534e-02,  6.5527e-02, -5.6064e-02,  1.5210e-01,
          1.8274e-01, -3.9153e-01,  1.9312e-01, -2.9789e-01, -8.3057e-02,
  

### Setup Model
Configure your experiment name and savepaths here, they will all be stored under the logs folder later on

In [3]:
import os
import torch
from diffusion.diffuser.utils import Trainer as dTrainer, Config as dConfig

exp_name = "test-walk-shuffled-128"
savepath = f'/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{exp_name}'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

if not os.path.exists(savepath):
    os.makedirs(savepath)
    os.makedirs(os.path.join(savepath, 'sampled_motions'))

In [4]:
from diffusion.diffuser.models.temporal_v2 import TemporalUnet 

horizon = dataset[0].trajectories.shape[0]
transition_dim = dataset[0].trajectories.shape[1]

model_config = dConfig(
    TemporalUnet,
    savepath=(savepath, 'model_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    cond_dim=transition_dim,
    device=device,
)
model = model_config()

/home/kenji/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



[utils/config ] Config: <class 'diffusion.diffuser.models.temporal_v2.TemporalUnet'>
    cond_dim: 69
    horizon: 32
    transition_dim: 69

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-walk-shuffled-128/model_config.pkl

[ models/temporal ] Channel dimensions: [(69, 128), (128, 256), (256, 512), (512, 1024)]
[(69, 128), (128, 256), (256, 512), (512, 1024)]


In [5]:
from diffusion.diffuser.models.diffusion_v2 import GaussianDiffusion

# model params, I am only using the very basic ones, some params are for conditioning
n_timesteps = 1000
loss_type = 'l2'
clip_denoised = False
predict_epsilon = False
action_weight = 5
loss_weights = None
loss_discount = 1
pos_dim = 35
vel_dim = 34

diffusion_config = dConfig(
    GaussianDiffusion,
    savepath=(savepath, "diffusion_config.pkl"),
    horizon=horizon,
    # transition_dim=transition_dim,
    observation_dim=pos_dim,
    action_dim=vel_dim,
    n_timesteps=n_timesteps,
    loss_type=loss_type,
    clip_denoised=clip_denoised,
    predict_epsilon=predict_epsilon,
    # loss weighting
    action_weight=action_weight,
    loss_weights=loss_weights,
    loss_discount=loss_discount,
    device=device,
)

diffusion = diffusion_config(model)


[utils/config ] Config: <class 'diffusion.diffuser.models.diffusion_v2.GaussianDiffusion'>
    action_dim: 34
    action_weight: 5
    clip_denoised: False
    horizon: 32
    loss_discount: 1
    loss_type: l2
    loss_weights: None
    n_timesteps: 1000
    observation_dim: 35
    predict_epsilon: False

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-walk-shuffled-128/diffusion_config.pkl



### Setup Training

In [6]:
learning_rate = 2e-4
gradient_accumulate_every = 2
ema_decay = 0.995
sample_freq = 2000
save_freq = 2000
n_train_steps = 1e5
n_saves = 5
save_parallel = False
bucket = None
n_reference = 8
train_batch_size = 32

trainer_config = dConfig(
    dTrainer,
    savepath=(savepath, 'trainer_config.pkl'),
    train_batch_size=train_batch_size,
    train_lr=learning_rate,
    gradient_accumulate_every=gradient_accumulate_every,
    ema_decay=ema_decay,
    sample_freq=sample_freq,
    save_freq=save_freq,
    label_freq=int(n_train_steps // n_saves),
    save_parallel=save_parallel,
    results_folder=savepath,
    bucket=bucket,
    n_reference=n_reference,
)

trainer = trainer_config(diffusion, dataset, renderer=None)


[utils/config ] Config: <class 'diffusion.diffuser.utils.training.Trainer'>
    bucket: None
    ema_decay: 0.995
    gradient_accumulate_every: 2
    label_freq: 20000
    n_reference: 8
    results_folder: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-walk-shuffled-128
    sample_freq: 2000
    save_freq: 2000
    save_parallel: False
    train_batch_size: 32
    train_lr: 0.0002

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-walk-shuffled-128/trainer_config.pkl



### Test if model and training loop works

In [7]:
import torch
from diffuser.utils import batchify

tunet = model
tunet.to(device)
test_data = dataset[0]
batch = batchify(test_data)
t = torch.randint(0, 1000, (1,), device=device).long().to(device)
res = tunet(batch.trajectories, cond=batch.conditions, time=t, verbose=True)
res, res.shape

x.shape torch.Size([1, 32, 69])
x.shape torch.Size([1, 69, 32])
x.shape torch.Size([1, 128, 32])
1
xfinal.shape torch.Size([1, 128, 16])
x.shape torch.Size([1, 256, 16])
2
xfinal.shape torch.Size([1, 256, 8])
x.shape torch.Size([1, 512, 8])
3
xfinal.shape torch.Size([1, 512, 4])
x.shape torch.Size([1, 1024, 4])
4
xfinal.shape torch.Size([1, 1024, 4])
xt1.shape torch.Size([1, 1024, 4])
xt2.shape torch.Size([1, 1024, 4])
xt3.shape torch.Size([1, 1024, 4])
pop.shape torch.Size([1, 1024, 4])
pop.shape torch.Size([1, 512, 8])
pop.shape torch.Size([1, 256, 16])


(tensor([[[ 0.1472, -0.2029,  0.0729,  ..., -0.0579,  0.3516,  0.2997],
          [-0.1191, -0.1982,  0.0128,  ..., -0.3008, -0.0494, -0.0490],
          [ 0.0971, -0.1143, -0.2023,  ..., -0.3670, -0.1985, -0.0176],
          ...,
          [-0.6000, -0.3256, -0.3277,  ...,  0.0972,  0.2678,  0.0975],
          [-0.1060,  0.1234, -0.2384,  ...,  0.0086, -0.0559, -0.3977],
          [-0.2736,  0.0179,  0.0779,  ...,  0.0876, -0.2222,  0.1691]]],
        device='cuda:0', grad_fn=<PermuteBackward0>),
 torch.Size([1, 32, 69]))

In [8]:
from diffuser.utils import report_parameters, batchify

report_parameters(model)

print('Testing forward...', end=' ', flush=True)
x = dataset[0]
batch = batchify(x)
loss, _ = diffusion.loss(batch.trajectories, {})
loss.backward()
print('✓')

[ utils/arrays ] Total parameters: 63.13 M
         downs.3.0.blocks.1.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.0.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.1.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         ups.0.0.blocks.0.block.0.weight: 5.24 M | Conv1d(2048, 512, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.0.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.1.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.0.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.1.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.0.b

### Train Model
It took me 80s to run 1 epoch and results were pretty good from just 1 epoch

In [10]:
n_steps_per_epoch = 1000
n_epochs = int(n_train_steps // n_steps_per_epoch)
print(n_epochs)
n_epochs = 3

for i in range(n_epochs):
    print(f'Epoch {i} / {n_epochs} | {savepath}')
    trainer.train(n_train_steps=n_steps_per_epoch)

trainer.save(n_epochs)

100
Epoch 0 / 3 | /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-walk-shuffled-128


100:   0.0479 | a0_loss:   0.0017 | t:  11.7294
200:   0.0347 | a0_loss:   0.0011 | t:  16.6243
300:   0.0059 | a0_loss:   0.0004 | t:  16.5960
400:   0.0199 | a0_loss:   0.0005 | t:  16.9970
500:   0.0161 | a0_loss:   0.0004 | t:  17.2997
600:   0.0292 | a0_loss:   0.0005 | t:  17.1336
700:   0.0234 | a0_loss:   0.0003 | t:  17.4197
800:   0.0305 | a0_loss:   0.0003 | t:  16.8901
900:   0.0330 | a0_loss:   0.0004 | t:  16.5573
1000:   0.0260 | a0_loss:   0.0005 | t:  16.7373
Epoch 1 / 3 | /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-walk-shuffled-128
1100:   0.0015 | a0_loss:   0.0001 | t:  11.4258
1200:   0.0091 | a0_loss:   0.0002 | t:  17.3797
1300:   0.0610 | a0_loss:   0.0007 | t:  16.6718
1400:   0.0291 | a0_loss:   0.0004 | t:  16.2923
1500:   0.0145 | a0_loss:   0.0002 | t:  16.7892
1600:   0.0027 | a0_loss:   0.0001 | t:  16.1838
1700:   0.0250 | a0_loss:   0.0004 | t:  16.3579
1800:   0.0238 | a0_loss:   0.0002 | t:  17.4345
1900:   0.0027 | a0_loss:   0.0001 | t:  1

### (Alternatively) load a checkpoint

In [7]:
# from diffusion.diffuser.utils import load_diffusion
# diffusion_experiment = load_diffusion(
#     "/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser", dataset=dataset, epoch=1)

# renderer = diffusion_experiment.renderer
# model = diffusion_experiment.trainer.ema_model

# Optionally load a checkpoint
trainer.load(3)
model = trainer.ema_model

### Sample from model - With goals to be at certain keyframes

Set the position data of certain keyframes to be along some trajectory

In [8]:
walk_traj = dataset[0].trajectories
walk_traj.unsqueeze_(0)
walk_traj.shape

torch.Size([1, 32, 69])

In [12]:
def trajectory_setting_fn(x):
    # pos 0, 1, 2 represents x, y and z

    # keyframe 1
    x[:, 0, 0] = 0
    x[:, 0, 1] = 0

    x[:, 1, 0] = 0.1
    x[:, 1, 1] = 0.1

    x[:, 2, 0] = 0.2
    x[:, 2, 1] = 0.2

    x[:, 3, 0] = 0.3
    x[:, 3, 1] = 0.3

    x[:, 4, 0] = 0.4
    x[:, 4, 1] = 0.4

    x[:, 5, 0] = 0.5
    x[:, 5, 1] = 0.5

    x[:, 6, 0] = 0.6
    x[:, 6, 1] = 0.6

    x[:, 7, 0] = 0.7
    x[:, 7, 1] = 0.7

    x[:, 8, 0] = 0.8
    x[:, 8, 1] = 0.8

    x[:, 9, 0] = 0.9
    x[:, 9, 1] = 0.9

    x[:, 10, 0] = 1.0
    x[:, 10, 1] = 1.0

    x[:, 11, 0] = 1.1
    x[:, 11, 1] = 1.1

    x[:, 12, 0] = 1.2
    x[:, 12, 1] = 1.2

    x[:, 13, 0] = 1.3
    x[:, 13, 1] = 1.3

    x[:, 14, 0] = 1.4
    x[:, 14, 1] = 1.4

    x[:, 15, 0] = 1.5
    x[:, 15, 1] = 1.5

    return x

In [13]:
from diffuser.utils import batchify
test = dataset[0]
batch = batchify(test)

sample = diffusion.p_sample_loop(batch.trajectories.shape, batch.conditions, conditioning_fn=trajectory_setting_fn)


                                                                                                    
1 / 1000 [                                                            ]   0% | 63.8 Hz
t : 999 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
2 / 1000 [                                                            ]   0% | 75.5 Hz
t : 998 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
3 / 1000 [                                                            ]   0% | 79.1 Hz
t : 997 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                              

### Save results to logs

In [14]:
import numpy as np


def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    pos_data = sample.trajectories.squeeze(0)[:, :35].cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion saved as {filename}")


save_motions(sample, f"{savepath}/sampled_motions", filename="trajectory-motion1.npy")

Motion saved as trajectory-motion1.npy


Alternatively, we can place checkpoints in the long cartwheel motion

In [24]:
from diffuser.utils import batchify

test = dataset[0]
batch = batchify(test)
multiplier = 3

consequent_cartwheels = dataset[0].trajectories.repeat(3, 1)
consequent_cartwheels = consequent_cartwheels.unsqueeze(0)
consequent_cartwheels.shape

torch.Size([1, 480, 35])

In [31]:
# phase_offset = sim_state.qpos[:3]
# phase_offset[2] = 0
consequent_cartwheels[0, 160, 6] = 0
consequent_cartwheels[0, 320, 6] = 0

Place the end cartwheel checkpoints in between the long motion

In [25]:
window = 50

x_t = torch.randn_like(consequent_cartwheels).to(device)
x_t[:, 160 - window : 160 + window, :] = consequent_cartwheels[
    :, 160 - window : 160 + window, :
]
x_t[:, 320 - window : 320 + window, :] = consequent_cartwheels[
    :, 320 - window : 320 + window, :
]

def apply_conditioning_checkpoints(x):
    x_t[:, 160 - window : 160 + window, :] = consequent_cartwheels[
        :, 160 - window : 160 + window, :
    ]
    x_t[:, 320 - window : 320 + window, :] = consequent_cartwheels[
        :, 320 - window : 320 + window, :
    ]
    return x

In [26]:
sample = diffusion.p_sample_loop(
    consequent_cartwheels.shape, batch.conditions, starting_motion=x_t, max_timesteps=500, conditioning_fn=apply_conditioning_checkpoints
)


                                                                                                    
1 / 500 [                                                            ]   0% | 6.0 Hz
t : 499 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
2 / 500 [                                                            ]   0% | 11.2 Hz
t : 498 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
3 / 500 [                                                            ]   0% | 15.8 Hz
t : 497 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                  

In [32]:
import numpy as np


def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    pos_data = sample.trajectories.squeeze(0).cpu().numpy()
    pos_data = consequent_cartwheels.squeeze(0).cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion saved as {filename}")


save_motions(sample, f"{savepath}/sampled_motions", filename="long-form-with-3-checkpoints-motion.npy")

Motion saved as long-form-with-3-checkpoints-motion.npy
